In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

# Static Configurations

> In this module, we configure stuff.

In [ ]:
#| default_exp configuration

## Imports

In [ ]:
#| export

from pathlib import Path
import thucy
from dotenv import set_key, load_dotenv, dotenv_values

## Configuration Class

The `Config` class manages all configuration through a combination of default values and environment variables loaded from `.env`.

In [ ]:
#| export

class Config:
    """Global configuration for the thucy project"""

    DEFAULTS = {
        "EXPERTS_MODEL": "gpt-5-mini",
        "LEAD_MODEL": "gpt-5",
        "SQL_EXPERT_MAX_TURNS": 30,
        "SCHEMA_EXPERT_MAX_TURNS": 30,
        "DATA_EXPERT_MAX_TURNS": 30,
        "GENAI_SERVER_URL": "http://127.0.0.1:5000",
    }
    
    def __init__(self):
        # Path Configs
        self.project_root = Path(thucy.__file__).parent.parent
        self.expertiments_dir = self.project_root / "experiments"
        self.results_dir = self.expertiments_dir / "results"  

        self.env_path = self.project_root / ".env"
        self.env_path.touch(exist_ok=True)

        # Load environment variables
        self._load_env()

    def _load_env(self):
        """Load everything in DEFAULTS + everything in the .env file."""
        # Load into the environment to make sure
        load_dotenv(self.env_path, override=True)

        env = dotenv_values(self.env_path)

        # defaults
        for key, default in self.DEFAULTS.items():
            val = env.get(key, default)
            if val.isdigit(): setattr(self, key.lower(), int(val))
            else: setattr(self, key.lower(), val)

        # user-defined keys (e.g., OPENAI_API_KEY)
        for key, val in env.items():
            if key not in self.DEFAULTS:
                if val.isdigit(): setattr(self, key.lower(), int(val))
                else: setattr(self, key.lower(), val)

    def set_env_var(self, key, value):
        """Set or update a variable and persist to .env."""
        value = str(value)
        set_key(self.env_path, key, value)
        setattr(self, key.lower(), value)
        self._load_env()

### Singleton Instance

We create a single global `config` instance that can be imported throughout the project.

In [ ]:
#| exports

config = Config()

### Usage Examples

The following cells demonstrate how to set and retrieve environment variables using the config system.

Check if variable exists before setting it:

In [ ]:
import os

In [ ]:
os.getenv("TEST_ENV_VAR1")

'thucy_is_great_001'

Set the variable and verify it's saved:

In [ ]:
config.set_env_var('TEST_ENV_VAR1', 'thucy_is_great_001')

In [ ]:
os.getenv("TEST_ENV_VAR1")

'thucy_is_great_001'

In [ ]:
config.test_env_var1

'thucy_is_great_001'

In [ ]:
config.set_env_var('TEST_ENV_VAR2', 33)

In [ ]:
os.getenv("TEST_ENV_VAR2")

'33'

Access via config object (lowercase attribute):

In [ ]:
config.test_env_var2

33

Update an existing variable:

In [ ]:
config.set_env_var('TEST_ENV_VAR2', 11)

In [ ]:
os.getenv("TEST_ENV_VAR2")

'11'

In [ ]:
config.test_env_var2

11

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()